In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPool2D
from tensorflow.keras.models import Sequential

# Get the data

In [ ]:
train_df = pd.read_csv('train.csv')
val_df = pd.read_csv('val.csv')
test_df = pd.read_csv('test.csv')

### Image preprocessing

In [ ]:
BATCH_SIZE = 32
IMG_HEIGHT = 180
IMG_WIDTH = 180

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  "training_set/training_set/",
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE,
  )

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  "training_set/training_set/",
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE,
  )

In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  "test_set/test_set/",
  #validation_split=0.2,
  #subset="test",
  #seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE,
  )

In [ ]:
class_names = train_ds.class_names
print(class_names)

For the training process we will use a total of 8005 images:


*   6404 for training
*   1601 for validation



### Data standardisation

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# normalization_layer = layers.Rescaling(1./255)

In [ ]:
# if ESTANDARIZAR_DATOS:
#   normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
#   image_batch, labels_batch = next(iter(normalized_ds))
#   first_image = image_batch[0]
#   # Notice the pixel values are now in `[0,1]`.
#   print(np.min(first_image), np.max(first_image))

# Model

In [ ]:
num_classes = len(class_names)

In [ ]:
custom_model = Sequential()

custom_model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = "relu", input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)))
custom_model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = "relu"))
custom_model.add(MaxPool2D(pool_size=(2,2)))

custom_model.add(Dropout(0.25))

custom_model.add(Conv2D(filters = 32, kernel_size = (3,3), activation = "relu"))
custom_model.add(Conv2D(filters = 32, kernel_size = (3,3), activation = "relu"))
custom_model.add(MaxPool2D(pool_size = (2,2)))

custom_model.add(Dropout(0.25))

custom_model.add(Conv2D(filters = 16, kernel_size = (3,3), activation = "relu"))
custom_model.add(Conv2D(filters = 16, kernel_size = (3,3), activation = "relu"))
custom_model.add(MaxPool2D(pool_size = (2,2)))

custom_model.add(Flatten())
custom_model.add(Dense(units = 12, activation = "relu"))
custom_model.add(Dense(units = 1, activation = "sigmoid"))

In [ ]:
# Make sure you have frozen the correct layers
for i, layer in enumerate(custom_model.layers):
    print(i, layer.name, layer.trainable)

In [ ]:
custom_model.summary()

In [ ]:
custom_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model_history = custom_model.fit(train_ds, validation_data = val_ds, epochs = 25)

In [ ]:
acc = model_history.history['accuracy']
val_acc = model_history.history['val_accuracy']

loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

epochs_range = range(25)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Model evaluation

In [ ]:
# Generate generalization metrics
score = custom_model.evaluate(test_ds, verbose=1)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')